# Factory pattern

In [1]:
import json
import os
import xml.etree.ElementTree as ET

from pathlib import Path

In [2]:
print(os.path.dirname(os.path.realpath("__file__")))

/mnt/Exdisk/git-cuongpiger/mastering-python-design-patterns/chap03


In [3]:
class JSONDataExtractor:
    def __init__(self, filepath: Path):
        self.data = {}
        with open(filepath) as f:
            self.data = json.load(f)

    @property
    def parsed_data(self):
        return self.data

In [4]:
class XMLDataExtractor:
    def __init__(self, filepath: Path):
        self.tree = ET.parse(filepath)

    @property
    def parsed_data(self):
        return self.tree

In [5]:
def extract_factory(filepath: Path):
    ext = filepath.name.split(".")[-1]
    if ext == "json":
        return JSONDataExtractor(filepath)
    elif ext == "xml":
        return XMLDataExtractor(filepath)
    else:
        raise ValueError("Cannot extract data")

In [6]:
def extract(case: str):
    dir_path = Path(os.path.realpath("__file__")).parent

    if case == "json":
        path = dir_path / Path("movies.json")
        factory = extract_factory(path)
        data = factory.parsed_data

        for movie in data:
            print(f"- {movie['title']}")
            director = movie["director"]
            if director:
                print(f"   Director: {director}")
            genre = movie["genre"]
            if genre:
                print(f"   Genre: {genre}")
    elif case == "xml":
        path = dir_path / Path("person.xml")
        factory = extract_factory(path)
        data = factory.parsed_data

        search_xpath = ".//person[lastName='Liar']"
        items = data.findall(search_xpath)
        for item in items:
            first = item.find("firstName").text
            last = item.find("lastName").text
            print(f"- {first} {last}")
            for pn in item.find("phoneNumbers"):
                pn_type = pn.attrib["type"]
                pn_val = pn.text
                phone = f"{pn_type}: {pn_val}"
                print(f"   {phone}")

In [7]:
print("* JSON case *")
extract(case="json")
print("* XML case *")
extract(case="xml")

* JSON case *
- After Dark in Central Park
- Boarding School Girls' Pajama Parade
- Buffalo Bill's Wild West Parad
- Caught
- Clowns Spinning Hats
- Capture of Boer Battery by British
   Director: James H. White
   Genre: Short documentary
- The Enchanted Drawing
   Director: J. Stuart Blackton
- Family Troubles
- Feeding Sea Lions
* XML case *
- Jimy Liar
   home: 212 555-1234
- Patty Liar
   home: 212 555-1234
   mobile: 001 452-8819


# Abstract factory pattern

In [13]:
class Frog:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def interact_with(self, obstacle):
        act = obstacle.action()
        msg = f"{self} the Frog encounters {obstacle} and {act}!"
        print(msg)

In [14]:
class FrogWorld:
    def __init__(self, name):
        print(self)
        self.player_name = name

    def __str__(self):
        return "\n\n\t------ Frog World -------"

    def make_character(self):
        return Frog(self.player_name)

    def make_obstacle(self):
        return Bug()

In [15]:
class Wizard:
    def __init__(self, name):
        self.name = name

    def __str__(self):
        return self.name

    def interact_with(self, obstacle):
        act = obstacle.action()
        msg = f"{self} the Wizard battles against {obstacle} and {act}!"
        print(msg)

In [16]:
class WizardWorld:
    def __init__(self, name):
        print(self)
        self.player_name = name

    def __str__(self):
        return "\n\n\t------ Wizard World -------"

    def make_character(self):
        return Wizard(self.player_name)

    def make_obstacle(self):
        return Ork()

In [17]:
class Ork:
    def __str__(self):
        return "an evil ork"

    def action(self):
        return "kills it"

In [18]:
class Bug:
    def __str__(self):
        return "a bug"

    def action(self):
        return "eats it"

In [19]:
class GameEnvironment:
    def __init__(self, factory):
        self.hero = factory.make_character()
        self.obstacle = factory.make_obstacle()

    def play(self):
        self.hero.interact_with(self.obstacle)

In [20]:
def validate_age(name):
    age = None
    try:
        age_input = input(f"Welcome {name}. How old are you? ")
        age = int(age_input)
    except ValueError:
        print(f"Age '{age_input}' is invalid, please try again...")
        return False, age
    return True, age

In [21]:
name = input("Hello. What's your name? ")
valid_input = False
while not valid_input:
    valid_input, age = validate_age(name)
game = FrogWorld if age < 18 else WizardWorld
environment = GameEnvironment(game(name))
environment.play()



	------ Wizard World -------
Cuong the Wizard battles against an evil ork and kills it!


In [22]:
name = input("Hello. What's your name? ")
valid_input = False
while not valid_input:
    valid_input, age = validate_age(name)
game = FrogWorld if age < 18 else WizardWorld
environment = GameEnvironment(game(name))
environment.play()



	------ Frog World -------
Van the Frog encounters a bug and eats it!
